# Function Calling with LangChain Agents

In this notebook, we will leverage [Agents](https://python.langchain.com/docs/modules/agents/) in LangChain (particularly, custom-defined agents) to demonstrate the capabilities of function-calling-augmented LLMs. 

In [1]:
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
    model_name="gpt-3.5-turbo"
)

## 1. Calculator Function

Computers can solve complex math problems, yet if we ask GPT-3.5 (or GPT-4, for that matter) for the answer to `(4.5*2.1)^2.2`, it fails.

In [4]:
print(llm.invoke("(4.5*2.1)^2.2").content)

Calculating the expression (4.5*2.1)^2.2:

(4.5*2.1)^2.2 = (9.45)^2.2
(9.45)^2.2 ≈ 94.5^2.2
94.5^2.2 ≈ 1,048.52

Therefore, (4.5*2.1)^2.2 is approximately 1,048.52.


To address this problem, we can leverage function calling to allow the LLM to use an actual calculator tool.

### 1.1 Creating a Custom Tool

We will create a tool that will evaluate basic arithmetic math problems. There many ways to define tools - we enumerate some of these approaches below.

#### 1.1.1 OOP Approach

We can define our own tools with an inheritance-based approach. 

In [5]:
from langchain.tools import BaseTool

class CalculatorTool(BaseTool):
    # The name and description are required by LangChain - they are used to help the LLM figure out what tool to use
    name = "Calculator"
    description = "A simple calculator tool that evaluates a mathemtical expression and returns the result as an integer. Do not pass untrusted input."
    
    def _run(self, input_expr: str) -> int:
        # Evaluate the mathematical expression to get the result
        # Replace ^ (mathematical exponent operator) with ** (python exponent operator)
        return eval(input_expr.replace("^", "**"), {}, {})

Note that the `eval` function is generally very dangerous (but for the sake of this example, it's fine). `eval(str)` evaluates the input string as a literal Python expression, running whatever is contained inside the string. For instance, `eval("4.5 * 2")` returns `9`, and `eval("[1, 2, 3]")` returns a Python list with the values 1, 2, and 3. However, what this means is that `eval("os.system("rm -rf /")")` can also be ran...

The two empty dictionaries we also pass as argument help to prevent some attacks, such as the `os.system` one; refer to the `eval` function's [documentation](https://realpython.com/python-eval-function/) to learn why. In practice, you would be best served using the `literal_eval` function from the `ast` library.

#### 1.1.2 Function Decorator Approach

Another method for custom tool definition is through the use of function [decorators](https://www.geeksforgeeks.org/decorators-in-python/) (a more advanced Python concept).

In [6]:
from langchain.tools import tool

@tool
def calculator(input_expr: str) -> int:
    """A simple calculator tool that evaluates a mathemtical expression and returns the result as an integer. Do not pass untrusted input."""
    return eval(input_expr.replace("^", "**"), {}, {})

#### Defining an Agent

In [7]:
# when giving tools to LLM, we must pass them as a list of tools
# tools = [CalculatorTool]
tools = [calculator]

In [8]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

zero_shot_agent("what is (4.5*2.1)^2.2?")

/Users/ernesto/umich/DSClub/W24-llm-augmentation/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/Users/ernesto/umich/DSClub/W24-llm-augmentation/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I need to calculate the result of the expression (4.5*2.1)^2.2
Action: calculator
Action Input: (4.5*2.1)^2.2
Observation: 139.94261298333066
Thought:I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

Note: If you are running into weird errors with the tools, try restarting your kernel.

### 1.2 Using an Existing Tool

LangChain's [LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html) would be a good solution here. The LLMMathChain turns math problems into Python Code (using the LLM) and runs the code to obtain the result. This would be very helpful for more complicated math problems.

In [9]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass them as a list of tools
tools = [math_tool]

/Users/ernesto/umich/DSClub/W24-llm-augmentation/venv/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [10]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I need to calculate the result of raising (4.5*2.1) to the power of 2.2.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer
Final Answer: 139.94

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94'}

## 2. Your Turn

Now, create a function calling application of your own. You are encouraged to use external APIs to create robust tools. Refer to `simple-weather-llm.py` for an example tool.

### The Idea: Philosophy Multiplexer 
A main bot will recieve input from a user, and then decide whether this question falls under the following categories:
- Epistemology -- https://plato.stanford.edu/entries/epistemology/
- Moral theory -- https://plato.stanford.edu/entries/moral-theory/
- Metaphysics -- https://plato.stanford.edu/entries/metaphysics/
- Linguistics -- https://plato.stanford.edu/entries/linguistics/
- Axiology -- https://plato.stanford.edu/entries/value-theory/
- Aesthetics -- https://plato.stanford.edu/entries/aesthetic-concept/
- Theology -- https://plato.stanford.edu/entries/philosophy-religion/
- Political philosophy (Focus on the concept of authority) -- https://plato.stanford.edu/entries/authority/
- 
It will provison an expert from each of the fields. Each expert will have access to a vector database where within i'll store the stanford encyclopedia of philosophy entry corresponding to that topic, as linked above.

##### Optional: figure out a way to carry on conversations
##### Optional^2: figure out a way to make these bots conversate with each other. E.g. 
```PMux -> Epistemology -> PMux -> Theology -> Pmux -> Axiology -> ...```

In [22]:
# Run to get updated encyclopedia entries
!./create_philosophy_directory.sh

The file philosophy_docs/epistemology.html already exists, skipping download.
The file philosophy_docs/moral-theory.html already exists, skipping download.
The file philosophy_docs/metaphysics.html already exists, skipping download.
The file philosophy_docs/linguistics.html already exists, skipping download.
The file philosophy_docs/axiology.html already exists, skipping download.
The file philosophy_docs/aesthetics.html already exists, skipping download.
The file philosophy_docs/theology.html already exists, skipping download.
The file philosophy_docs/political-philosophy.html already exists, skipping download.


In [23]:
class MyTool(BaseTool):
    name = "The name of the tool"
    description = "A description of the tool"

    # change the arguments as needed
    def _run(self, **kwargs):
        return "Hello, World!"

In [39]:
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import tiktoken

In [40]:
aesthetics_loader = UnstructuredHTMLLoader("philosophy_docs/aesthetics.html")
axiology_loader = UnstructuredHTMLLoader("philosophy_docs/axiology_loader.html")
epistemology_loader = UnstructuredHTMLLoader("philosophy_docs/epistemology.html")
linguistics_loader = UnstructuredHTMLLoader("philosophy_docs/linguistics.html")
metaphysics_loader = UnstructuredHTMLLoader("philosophy_docs/metaphysics.html")
moraltheory_loader = UnstructuredHTMLLoader("philosophy_docs/moral-theory.html")
politicalphilosophy_loader = UnstructuredHTMLLoader("philosophy_docs/political-philosophy.html")
theology_loader = UnstructuredHTMLLoader("philosophy_docs/theology.html")

In [41]:
tokenizer = tiktoken.get_encoding("cl100k_base")

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [42]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

In [ ]:
chunks = text_splitter.split_text()